# Setup

## Imports

In [1]:
import os
import itertools

from glob import glob
from skimage.transform import resize
from tqdm import tqdm_notebook

vai Modules

In [2]:
from vaiutils import path_consts, plot_images, summarize_tensor, randpick

Keras Modules

In [3]:
import keras.backend as K

Using TensorFlow backend.
/home/vaisakh/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Define Useful Features

In [4]:
for k, v in path_consts('COCO'):
    exec(k + ' = v')

Custom Modules

In [5]:
from colorizer import rgb_to_lab, lab_to_rgb

In [6]:
pixel = K.placeholder((None, None, 3))
rgb2lab = K.Function([pixel], [rgb_to_lab(pixel)])
lab2rgb = K.Function([pixel], [lab_to_rgb(pixel)])

In [7]:
def encode_ab(ab):
    a, b = np.take(ab, 0, -1).astype(uint8), np.take(ab, 1, -1).astype(uint8)
    return (a + 128) + (b + 128) * 256

## Checks

In [8]:
assert os.path.exists(os.path.join(DIR_CHECKPOINTS, 'ab_bins.npy')), 'Gamut Bins not created/found.\nRun GamutBins Notebook first'

## Load Data

In [9]:
ab_bins = np.load(os.path.join(DIR_CHECKPOINTS, 'ab_bins.npy'))
Q = len(ab_bins)

In [10]:
filenames = glob(os.path.join(DIR_DATA, 'val2017', '*.jpg'))

# Extract Probabilities

In [86]:
ab = np.stack(itertools.product(*[np.arange(-128, 128)]*2))

In [87]:
ab_idx = encode_ab(ab)

In [88]:
unique_idx = np.unique(ab_idx, return_counts=True)
assert np.all(np.sort(unique_idx[0]) == np.arange(2**16))
assert np.all(unique_idx[1] == 1)
del unique_idx

In [89]:
ab = np.expand_dims(ab[ab_idx], -1)

In [91]:
distances = np.linalg.norm(ab - ab_bins.T, axis=-2)

In [90]:
sigma = 5

In [92]:
probabilities = K.eval(K.softmax(-cp.asnumpy(distances) ** 2 / (2 * sigma ** 2)))

In [22]:
probabilities = np.sort(probabilities)

In [25]:
probabilities[:, :-5] = 0

In [103]:
assert np.all(np.abs(probabilities.sum(1) - 1) < 1e-12)

In [26]:
np.save(os.path.join(DIR_CHECKPOINTS, 'probabilities'), probabilities)

# Encode Images

In [167]:
os.makedirs(os.path.join(DIR_DATA, 'lab'), exist_ok=True)

In [177]:
for filename in tqdm_notebook(filenames):
    img = resize(imread(filename), (224, 224), mode='constant')

    lab = rgb2lab([img])[0].astype(int8)
    l, ab = lab[:, :, :1], lab[:, :, 1:]

    l_uint16 = l.astype(uint16)
    assert np.all(l_uint16 == l)

    lab = np.append(l_uint16, np.expand_dims(encode_ab(ab), -1), -1)

    file_id = os.path.split(filename)[1][:-4]
    np.save(os.path.join(DIR_DATA, 'lab', file_id), lab)

HBox(children=(IntProgress(value=0, max=4863), HTML(value='')))

In [181]:
filenames = glob(os.path.join(DIR_DATA, 'lab', '*.npy'))

In [199]:
probabilities = K.constant(probabilities)

In [202]:
for filename in tqdm_notebook(filenames):
    lab = K.constant(np.load(filename), dtype=int32)
    probs = K.gather(probabilities, lab[:, :, 1])

HBox(children=(IntProgress(value=0, max=4863), HTML(value='')))